In [ ]:
import pandas as pd, json

df = pd.read_csv("/content/neurocore_3state_dataset_llm_ready.csv")

with open("/content/neurocore_llm.jsonl", "w") as f:
    for _, row in df.iterrows():
        item = {
            "instruction": f"Explain the neural dynamics for mean_r={row.mean_r:.3f}, mean_V={row.mean_V:.3f}, state={row.state}",
            "response": row.advice
        }
        f.write(json.dumps(item) + "\n")

print("✅ neurocore_llm.jsonl ready!")


✅ neurocore_llm.jsonl ready!


In [ ]:
from google.colab import files

files.download('neurocore_llm.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Trainer, TrainingArguments, DataCollatorForSeq2Seq
)
from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files="/content/neurocore_llm.jsonl")["train"]

print("📊 Sample data:")
print(dataset[0])

# Load tokenizer + model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Verify column names
print("🧾 Columns:", dataset.column_names)

# Tokenization function
def preprocess(examples):
    inputs = [f"Explain the neural dynamics for: {x}" for x in examples["instruction"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["response"], max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Updated arguments
args = TrainingArguments(
    output_dir="/content/neurocore_explainer_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=1e-4,  # slightly higher
    logging_steps=50,
    fp16=False,          # disable half-precision on small models
    report_to="none",    # no wandb
    save_total_limit=1,
)

trainer = Trainer(model=model, args=args, train_dataset=dataset, data_collator=collator)

trainer.train()

trainer.save_model("/content/neurocore_explainer_model")
tokenizer.save_pretrained("/content/neurocore_explainer_model")

print("✅ Model fine-tuned successfully and saved at /content/neurocore_explainer_model")


Generating train split: 0 examples [00:00, ? examples/s]

📊 Sample data:
{'instruction': 'Explain the neural dynamics for mean_r=0.670, mean_V=-1.225, state=hyperactive_or_excited', 'response': 'The network exhibits excessive neural excitation and high firing rates. This pattern suggests strong coupling or reduced inhibition, leading to hyperactive dynamics. It is recommended to decrease excitatory coupling strength (k) or increase inhibitory influence (η) to restore balanced activity.'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

🧾 Columns: ['instruction', 'response']


Map:   0%|          | 0/1064 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Step,Training Loss
50,2.052900
100,0.156500
150,0.026300
200,0.014200
250,0.009800
300,0.008000
350,0.006300


✅ Model fine-tuned successfully and saved at /content/neurocore_explainer_model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="/content/neurocore_explainer_model", device=0)

prompt = "Explain the neural dynamics for mean_r=0.45, mean_V=-2.8, state=normal"
result = pipe(prompt, max_new_tokens=150)[0]["generated_text"]

print("🧠", result)


Device set to use cuda:0


🧠 The network is operating in a stable and balanced regime. Neural firing rates and membrane potentials are within the normal physiological range, indicating healthy signal transmission and proper excitatory-inhibitory balance. No modulation or external adjustment is required.


In [ ]:
examples = [
    {"mean_r": 0.12, "mean_V": -4.6, "state": "suppressed_or_inhibited"},
    {"mean_r": 0.71, "mean_V": -1.7, "state": "hyperactive_or_excited"}
]

for e in examples:
    prompt = f"Explain the neural dynamics for mean_r={e['mean_r']}, mean_V={e['mean_V']}, state={e['state']}"
    out = pipe(prompt, max_new_tokens=150)[0]['generated_text']
    print(f"\n🧩 {prompt}\n💡 {out}")



🧩 Explain the neural dynamics for mean_r=0.12, mean_V=-4.6, state=suppressed_or_inhibited
💡 The network shows abnormally low firing activity, indicating suppressed or inhibited dynamics. This can arise from excessive inhibitory input or insufficient external drive. Increasing excitatory current (I_ext) or reducing inhibitory parameters could help stabilize and re-activate the network.

🧩 Explain the neural dynamics for mean_r=0.71, mean_V=-1.7, state=hyperactive_or_excited
💡 The network exhibits excessive neural excitation and high firing rates. This pattern suggests strong coupling or reduced inhibition, leading to hyperactive dynamics. It is recommended to decrease excitatory coupling strength (k) or increase inhibitory influence () to restore balanced activity.


In [ ]:
import shutil
shutil.make_archive("/content/neurocore_explainer_model", 'zip', "/content/neurocore_explainer_model")

from google.colab import files
files.download("/content/neurocore_explainer_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>